# nucleus

> core functionalities of this lib

In [ ]:
#| default_exp nucleus

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export

        

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()